In [1]:
'''
1. 将拼多多收入与订单合并
2. 对收入分别按订单汇总、按存货编码汇总
'''

import easygui
import os
import pandas as pd
import re
import numpy as np



In [2]:
#所有订单
fname_dingdan = r"F:\a00nutstore\008\zw08\电商\拼多多\100306881orders_export2024-09-30-16-58-49.csv"
df_dingdan0 = pd.read_csv(fname_dingdan)
df_dingdan0.head()

,商品,订单号,订单状态,商品数量(件),是否审核中,支付时间,承诺发货时间,承诺送达时间,发货时间,确认收货时间,...,预约配送时间,是否节能补贴,是否分期,分期期数,手续费承担方,是否无痕发货,订单来源,是否社区团购,小区送达情况,分期方式
0,500页加厚英语定位练习纸本英语定位纸英文专用纸小学初中高中,240929-658935703962749,已发货，待收货,1,正常,2024/9/29 23:49,2024/10/1 23:49,NaN,2024/9/30 11:14,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
1,500页加厚英语定位练习纸本英语定位纸英文专用纸小学初中高中,240929-379762829722749,已发货，待收货,1,正常,2024/9/29 23:49,2024/10/1 23:49,NaN,2024/9/30 11:14,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
2,莱特5本加厚200页B5学生笔记本办公记事本白领工作记录本子笔记本,240929-211738969641353,已发货，待收货,1,正常,2024/9/29 23:44,2024/10/1 23:44,NaN,2024/9/30 11:14,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
3,莱特5本加厚200页B5学生笔记本办公记事本白领工作记录本子笔记本,240929-611277893212149,已发货，待收货,1,正常,2024/9/29 23:37,2024/10/1 23:37,NaN,2024/9/30 11:14,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
4,莱特5本加厚200页B5学生笔记本办公记事本白领工作记录本子笔记本,240929-080425849263824,未发货，退款成功,1,正常,2024/9/29 23:34,2024/10/1 23:34,NaN,\t,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN


In [4]:
df_dingdan0.columns.to_list()

['商品',
 '订单号',
 '订单状态',
 '商品数量(件)',
 '是否审核中',
 '支付时间',
 '承诺发货时间',
 '承诺送达时间',
 '发货时间',
 '确认收货时间',
 '商品id',
 '商品规格',
 '样式ID',
 '商品一级类目',
 '商品二级类目',
 '商品三级类目',
 '商品四级类目',
 '商家编码-规格维度',
 '商家编码-商品维度',
 '商家备注',
 '买家留言',
 '售后状态',
 '消费者资料',
 '订单成交时间',
 '消费者资料上传时间',
 '赠品名称',
 '赠品id',
 '赠品规格',
 '赠品数量',
 '团ID',
 '团身份',
 '是否已开始配送',
 '配送员是否已送达',
 '配送状态',
 '同城是否普通快递',
 '商品总价(元)',
 '邮费(元)',
 '店铺优惠折扣(元)',
 '平台优惠折扣(元)',
 '多多支付立减金额(元)',
 '上门安装费(元)',
 '送货入户费(元)',
 '送货入户并安装费(元)',
 '用户实付金额(元)',
 '商家实收金额(元)',
 '快递单号',
 '快递公司',
 '省',
 '市',
 '区',
 '街道/镇',
 '配送员手机号',
 '是否抽奖或0元试用',
 '是否顺丰加价',
 '用户购买手机号',
 '旅行类信息',
 '充值号码',
 '是否门店自提',
 '门店名称',
 '门店自定义编码',
 '关联货品编码',
 '货品名称',
 '货品类型',
 '子货品',
 '仓库名称',
 '仓库所在地址',
 '海淘清关订单号',
 '支付ID',
 '支付方式',
 '保税仓',
 '是否直播间成交',
 '是否直播间引导成交',
 '预约配送时间',
 '是否节能补贴',
 '是否分期',
 '分期期数',
 '手续费承担方',
 '是否无痕发货',
 '订单来源',
 '是否社区团购',
 '小区送达情况',
 '分期方式']

In [5]:
dic = {'五': 5,
       '六': 6,
       '七': 7,
       '八': 8,
       '九': 9,
       '十': 10,
       '四': 4,
       '三': 3,
       '二': 2,
       '一': 1}


def addBen(string):
    # string = r'(81本)'
    pattern1 = r'(?P<ben>\d+)本'
    pattern2 = r'(?P<ben>[十九八七六五四三二一])本'
    regexp1 = re.compile(pattern1)
    mat1 = regexp1.search(string)
    regexp2 = re.compile(pattern2)
    mat2 = regexp2.search(string)
    if mat1:
        ben = int(mat1.group('ben'))
    else:
        if mat2:
            ben = dic.get(mat2.group('ben'))
        else:
            ben = 1
    return ben


# df1 = df1[lst]
# df1['含量'] = df1['规格名称'].map(addBen)
# df1 = df1.assign(本数=df1['数量'] * df1['含量'])

In [ ]:
df_dingdan1.columns.to_list()

In [8]:
df_dingdan1 = df_dingdan0[['订单号','发货时间','商家编码-规格维度','商家实收金额(元)','商品数量(件)','商品规格']]
df_dingdan1.columns = ['dingdanhao','riqi','code','jiner','shuliang','guige']
df_dingdan1[df_dingdan1['dingdanhao'].isnull()]
df_dingdan1['hanliang'] = df_dingdan1['guige'].map(addBen)
df_dingdan1 = df_dingdan1.assign(shuliang=df_dingdan1['shuliang'] * df_dingdan1['hanliang'])
df_dingdan1.head()

C:\Users\redda\AppData\Local\Temp\ipykernel_27672\402509756.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dingdan1['hanliang'] = df_dingdan1['guige'].map(addBen)


,dingdanhao,riqi,code,jiner,shuliang,guige,hanliang
0,240929-658935703962749,2024/9/30 11:14,EJDWB550-英高,7.84,5,"高级版,5本",5
1,240929-379762829722749,2024/9/30 11:14,EJDWB550-英初,7.83,5,"初级版,5本",5
2,240929-211738969641353,2024/9/30 11:14,N0581,7.85,5,B5五本（每本40张）,5
3,240929-611277893212149,2024/9/30 11:14,N0483,9.57,5,A5五本（每本80张）,5
4,240929-080425849263824,\t,N0481,4.27,3,A5三本（每本40张）,3


In [13]:
df_dingdan2 = df_dingdan1.iloc[:,:5]
df_dingdan2 = df_dingdan2.set_index('dingdanhao')
df_dingdan2.head()

,riqi,code,jiner,shuliang
dingdanhao,,,,
240929-658935703962749,2024/9/30 11:14,EJDWB550-英高,7.84,5
240929-379762829722749,2024/9/30 11:14,EJDWB550-英初,7.83,5
240929-211738969641353,2024/9/30 11:14,N0581,7.85,5
240929-611277893212149,2024/9/30 11:14,N0483,9.57,5
240929-080425849263824,\t,N0481,4.27,3


In [31]:
#所有收入
fname_shouru = r"F:\a00nutstore\008\zw08\电商\拼多多\pdd-mall-bill-detail(100306881)_2024-09-30-07-09-49_3.csv"
df_shouru0 = pd.read_csv(fname_shouru,encoding = 'gb18030')
df_shouru0

,商户订单号,发生时间,收入金额（+元）,支出金额（-元）,账务类型,备注,业务描述
0,240928-589247332703567,2024/9/28 23:45,0.07,0.00,技术服务费,基础技术服务费返还,0030002|技术服务费-基础技术服务费
1,240928-589247332703567,2024/9/28 23:45,0.00,-10.48,退款,-,0020002|交易退款-订单退款
2,240928-589247332703567,2024/9/28 23:45,0.00,-1.82,优惠券结算,退款扣除优惠券支付金额,0020005|交易退款-优惠券结算
3,240928-589247332703567,2024/9/28 23:45,10.48,0.00,交易收入,-,0010002|交易收入-订单收入
4,240928-589247332703567,2024/9/28 23:45,1.82,0.00,优惠券结算,交易成功优惠券支付金额结算,0010005|交易收入-优惠券结算
...,...,...,...,...,...,...,...
64699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64700,#支出合计：38362笔-271037.53元,NaN,NaN,NaN,NaN,NaN,NaN
64701,#收入合计：26337笔245178.31元,NaN,NaN,NaN,NaN,NaN,NaN
64702,#总计：-25859.22元,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df_shouru1 = df_shouru0.dropna(thresh = 3)
regax = r'（订单(?P<dingdanhao>\d{6}-\d{15})）'
pattern = re.compile(regax)
# mat = pattern.search(string)
def getDingdanhao(s):
    mat = pattern.search(s)
    if mat :
        s = mat.group('dingdanhao')
    else :
        s = ''
    return s

In [ ]:
# string = '2024-09-28售后补偿现金券扣款（订单240923-652476454442206）'

# pattern = re.compile(regax)
# mat = pattern.search(string)
# if mat :
#     dingdanhao = mat.group('dingdanhao')
# else :
#     pass

In [47]:
df_shouru1['商户订单号'].dtype

dtype('O')

In [45]:
df_shouru1['商户订单号']= df_shouru1['商户订单号'].mask(df_shouru1['商户订单号'].isnull(),df_shouru0['备注'].map(getDingdanhao))
df_shouru1.columns = ['dingdanhao', 'riqi', 'ru', 'chu', 'leixing', 'beizhu', 'miaoshu']
df_shouru2 = df_shouru1.loc[df_shouru1.leixing != '提现']
df_shouru2 = df_shouru2.assign(shouru = df_shouru2['ru'] - df_shouru2['chu'])
df_shouru2['dingdanhao'] = df_shouru2['dingdanhao'].fillna('888888-888888888888888')
shouru_pivot =df_shouru2.pivot_table(index = 'dingdanhao',values =['ru','chu','shouru'],aggfunc = np.sum)
shouru_pivot


TypeError: expected string or bytes-like object, got 'float'

In [34]:
result0 =pd.merge(shouru_pivot,df_dingdan2,left_index = True,right_index = True,how = 'outer')
result0

,chu,ru,shouru,riqi,code,jiner,shuliang
dingdanhao,,,,,,,
,-1112.38,18.32,1130.7,NaN,NaN,NaN,NaN
240625-246792907173009,-9.30,0.00,9.3,NaN,NaN,NaN,NaN
240702-000068165411457,NaN,NaN,NaN,2024/7/2 12:17,N0781(线条),5.90,5.0
240702-002668706081035,NaN,NaN,NaN,2024/7/2 8:15,N0781(方格),5.85,5.0
240702-012284137890533,NaN,NaN,NaN,2024/7/3 8:07,N0482,8.10,5.0
...,...,...,...,...,...,...,...
240929-678994941551326,NaN,NaN,NaN,\t,N0582,10.90,5.0
240929-682077734432425,NaN,NaN,NaN,2024/9/29 15:43,RFZW-A460N（500格）,15.50,5.0
240929-683021473293766,NaN,NaN,NaN,2024/9/29 11:48,N0481,4.27,3.0


In [35]:
#空值填充为0
for i in ['chu','ru','shouru','jiner','shuliang']:
    result0[i] = result0[i].fillna(0)
#code填充
result0['code'] = result0['code'].fillna('8888')

#日期填充
result0['riqi'] = result0['riqi'].replace('\t',None)
result0['riqi'] = result0['riqi'].ffill()
result0['riqi'] = result0['riqi'].bfill()
result0

,chu,ru,shouru,riqi,code,jiner,shuliang
dingdanhao,,,,,,,
,-1112.38,18.32,1130.7,2024/7/2 12:17,8888,0.00,0.0
240625-246792907173009,-9.30,0.00,9.3,2024/7/2 12:17,8888,0.00,0.0
240702-000068165411457,0.00,0.00,0.0,2024/7/2 12:17,N0781(线条),5.90,5.0
240702-002668706081035,0.00,0.00,0.0,2024/7/2 8:15,N0781(方格),5.85,5.0
240702-012284137890533,0.00,0.00,0.0,2024/7/3 8:07,N0482,8.10,5.0
...,...,...,...,...,...,...,...
240929-678994941551326,0.00,0.00,0.0,2024/9/29 15:43,N0582,10.90,5.0
240929-682077734432425,0.00,0.00,0.0,2024/9/29 15:43,RFZW-A460N（500格）,15.50,5.0
240929-683021473293766,0.00,0.00,0.0,2024/9/29 11:48,N0481,4.27,3.0


In [36]:
result0.to_excel('分配前outer-2-4.xlsx')

In [37]:
df_dingdan2.to_excel('订单号-4.xlsx')

In [ ]:
# left = pd.read_clipboard()
# right = pd.read_clipboard()
# test_result = pd.merge(left = left,right = right,how = 'outer')
# test_result = pd.merge(right = right,left = left,how = 'outer')
# test_result 

In [38]:
gp = result0.groupby('dingdanhao')
gp

In [39]:
#将订单总金额按各产品的明细销售额进行分配
data = []
for k,v in gp:
    # print(k,v)
    mean = v.shouru.mean()
    total = v['jiner'].sum()
    # print('mean',mean)
    # print('total',total)
    v['result_jiner'] = mean/total*(v.jiner)
    data.append(v)
result1 = pd.concat(data)
result1
    

C:\Users\redda\AppData\Local\Temp\ipykernel_27672\2925582487.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  v['result_jiner'] = mean/total*(v.jiner)
C:\Users\redda\AppData\Local\Temp\ipykernel_27672\2925582487.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  v['result_jiner'] = mean/total*(v.jiner)


,chu,ru,shouru,riqi,code,jiner,shuliang,result_jiner
dingdanhao,,,,,,,,
,-1112.38,18.32,1130.7,2024/7/2 12:17,8888,0.00,0.0,NaN
240625-246792907173009,-9.30,0.00,9.3,2024/7/2 12:17,8888,0.00,0.0,NaN
240702-000068165411457,0.00,0.00,0.0,2024/7/2 12:17,N0781(线条),5.90,5.0,0.0
240702-002668706081035,0.00,0.00,0.0,2024/7/2 8:15,N0781(方格),5.85,5.0,0.0
240702-012284137890533,0.00,0.00,0.0,2024/7/3 8:07,N0482,8.10,5.0,0.0
...,...,...,...,...,...,...,...,...
240929-678994941551326,0.00,0.00,0.0,2024/9/29 15:43,N0582,10.90,5.0,0.0
240929-682077734432425,0.00,0.00,0.0,2024/9/29 15:43,RFZW-A460N（500格）,15.50,5.0,0.0
240929-683021473293766,0.00,0.00,0.0,2024/9/29 11:48,N0481,4.27,3.0,0.0


In [40]:
result1.to_excel('分配后4.xlsx')